In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics.engine.results import Results
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [2]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
    "path" : str(DATASETS_DIR.absolute()),
    'train': str(TRAIN_IMAGES_DIR.absolute()),
    'val': str(VAL_IMAGES_DIR.absolute()),
    'test': str(TEST_IMAGES_DIR.absolute()),
    'nc': num_classes,
    'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [3]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("runs/detect/train*/weights/best.pt"))[-1]
BEST_PATH

'runs/detect/train5/weights/best.pt'

In [4]:
for flag, v in ExifTags.TAGS.items():
    if v == "Orientation":
        break


def load_image(filepath):
    image = Image.open(filepath)

    exif = image._getexif()
    if exif is None:
        return image

    orientation_value = exif.get(flag, None)

    if orientation_value == 3:
        image = image.rotate(180, expand=True)
    elif orientation_value == 6:
        image = image.rotate(270, expand=True)
    elif orientation_value == 8:
        image = image.rotate(90, expand=True)
    return image


flag

274

In [5]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# Path to the test images directory
test_dir_path = TEST_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = os.listdir(test_dir_path)

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []

# Iterate through each image in the directory
for image_file in tqdm(image_files):
    # Full path to the image
    img_path = os.path.join(test_dir_path, image_file)

    # Make predictions on the image
    results: list[Results] = model(
        load_image(img_path),
        conf=0.0,
        imgsz=1024,
        max_det=30,
        verbose=False,
    )

    # Extract bounding boxes, confidence scores, and class labels
    boxes = (
        results[0].boxes.xyxy.tolist() if results[0].boxes else []
    )  # Bounding boxes in xyxy format
    classes = results[0].boxes.cls.tolist() if results[0].boxes else []  # Class indices
    confidences = (
        results[0].boxes.conf.tolist() if results[0].boxes else []
    )  # Confidence scores
    names = results[0].names  # Class names dictionary

    if boxes:  # If detections are found
        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box
            detected_class = names[
                int(cls)
            ]  # Get the class name from the names dictionary

            # Add the result to the all_data list
            all_data.append(
                {
                    "Image_ID": str(image_file),
                    "class": detected_class,
                    "confidence": conf,
                    "ymin": y1,
                    "xmin": x1,
                    "ymax": y2,
                    "xmax": x2,
                }
            )
    else:  # If no objects are detected
        all_data.append(
            {
                "Image_ID": str(image_file),
                "class": None,
                "confidence": None,
                "ymin": None,
                "xmin": None,
                "ymax": None,
                "xmax": None,
            }
        )

100%|██████████| 1626/1626 [03:31<00:00,  7.70it/s]


In [6]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [7]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_cWEAQI.jpeg,healthy,0.725997,65.496201,3.566622,3809.929688,1677.347778
1,ID_cWEAQI.jpeg,healthy,0.087389,187.336441,5.244195,1441.073364,472.208466
2,ID_cWEAQI.jpeg,healthy,0.012052,245.344498,11.928021,1187.613770,593.236511
3,ID_cWEAQI.jpeg,healthy,0.008435,143.887268,4.444718,1956.317017,531.991089
4,ID_cWEAQI.jpeg,healthy,0.005113,338.875488,1350.859375,1167.488037,1797.457764


In [8]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,48780.000000,48780.000000,48780.000000,48780.000000,48780.000000
mean,0.046722,638.084714,643.439096,1448.241103,1258.090383
std,0.150417,872.602811,817.941180,1152.983540,1007.268527
min,0.000059,0.000000,0.000000,7.934352,12.008354
25%,0.001457,0.479921,0.446976,490.952324,416.000000
50%,0.003626,220.597099,316.739685,1101.279175,959.972443
75%,0.011210,971.389771,937.063843,2068.005920,1839.627808
max,0.913472,3936.517578,3974.330322,4128.000000,4128.000000


In [9]:
sub['class'].value_counts()

class
healthy        17787
cssvd          17571
anthracnose    13422
Name: count, dtype: int64

In [10]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64

class
healthy        1153
cssvd           801
anthracnose     694
None             57
Name: count, dtype: int6

In [11]:
sub.to_csv("dataset/predictions/02-predictions.csv", index=False)

In [12]:
sub["confidence"].describe()

count    48780.000000
mean         0.046722
std          0.150417
min          0.000059
25%          0.001457
50%          0.003626
75%          0.011210
max          0.913472
Name: confidence, dtype: float64

In [13]:
import pandas as pd

sub = pd.read_csv('dataset/predictions/02-predictions.csv')

sub.sample(6)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
33626,ID_y1htIj.jpeg,anthracnose,0.001147,2510.420898,3.925003,3359.287598,2196.992676
21564,ID_r5wbRA.jpeg,healthy,0.003873,0.272049,113.893456,295.757812,1547.685425
43632,ID_rGRsVE.jpg,cssvd,0.001658,0.150242,386.230164,297.761475,959.511108
28734,ID_pvsYXM.jpeg,cssvd,0.000425,160.695541,277.496796,1268.076050,1486.927124
12464,ID_TkQFgc.jpg,healthy,0.013485,0.000000,0.000000,442.127350,115.041840
2444,ID_slp0C2.jpg,anthracnose,0.002921,12.973633,0.000000,1532.167725,605.140503


In [14]:
sub["Image_ID"].value_counts().describe()

count    1626.0
mean       30.0
std         0.0
min        30.0
25%        30.0
50%        30.0
75%        30.0
max        30.0
Name: count, dtype: float64

In [15]:
sub["Image_ID"].nunique()

1626

In [16]:
sub.isna().sum()

Image_ID      0
class         0
confidence    0
ymin          0
xmin          0
ymax          0
xmax          0
dtype: int64